# Cleaning and Examining Data

In [4]:
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

df = pandas.DataFrame.from_csv('train.tsv', sep='\t')

In [5]:
# looks for any commas, periods, and apostrophes with any leading or following spaces
remove_punct = r'(( )*((\.)|(\'s*)|(,))( )*)'

# remove leading and trailing spaces
remove_spaces = r'(^( )*)|(( )*$)'

# replaces the pattern mentioned above with a single space
df['Phrase'] = df['Phrase'].str.replace(remove_punct, ' ')
df = df[df['Phrase'] != ' ']
df['Phrase'] = df['Phrase'].str.replace(remove_spaces, '')

In [6]:
B = []
before = ""
for r in df['Phrase']:
    if r != before:
        B.append(True)
        before = r
    else:
        B.append(False)
df = df[B]

In [7]:
i = 0
for col in df['Phrase'].head(80):
    print i, col
    print
    i += 1

0 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amounts to much of a story

1 A series of escapades demonstrating the adage that what is good for the goose

2 A series

3 A

4 series

5 of escapades demonstrating the adage that what is good for the goose

6 of

7 escapades demonstrating the adage that what is good for the goose

8 escapades

9 demonstrating the adage that what is good for the goose

10 demonstrating the adage

11 demonstrating

12 the adage

13 the

14 adage

15 that what is good for the goose

16 that

17 what is good for the goose

18 what

19 is good for the goose

20 is

21 good for the goose

22 good

23 for the goose

24 for

25 the goose

26 goose

27 is also good for the gander some of which occasionally amuses but none of which amounts to much of a story

28 is also

29 also

30 good for the gander some of which occasionally amuses but none of which 

In [8]:
class Node:
    def __init__(self, phrase, parent, left, right):
        self.parent = parent
        self.left = left
        self.right = right
        self.phrase = phrase
    def get_left(self):
        return self.left
    def get_right(self):
        return self.right
    def get_parent(self):
        return self.parent
    def set_left(self, left):
        self.left = left
    def set_right(self, right):
        self.right = right

In [9]:
root = Node("This quiet introspective and entertaining independent is worth seeking", 0, 0, 0)
phr = []
top80 = df['Phrase'].head(80)
j = 0
for r in top80:
    if j > 59 and j <= 74:
        phr.append(r)
    j += 1
print phr

['This quiet introspective and entertaining independent', 'This', 'quiet introspective and entertaining independent', 'quiet introspective and entertaining', 'quiet', 'introspective and entertaining', 'introspective and', 'introspective', 'and', 'entertaining', 'independent', 'is worth seeking', 'is worth', 'worth', 'seeking']


In [10]:
import pydot

colors = {1: "#F183B3", 2: "#E26E6E", 3: "#DB1F2A", 4: "#FF1100", 5: "#CA002A"}

def make_phrase_ptree(root, phrases, graph):
    if len(phrases) == 0:
        print "error: ran out of phrases"
        return
    if ' ' not in root.phrase:
        f(root.parent, phrases, graph)
    elif root.left == 0:
        root.left = Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        f(root.left, phrases[1:], graph)
    elif root.right == 0:
        root.right = Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        f(root.right, phrases[1:], graph)
    elif root.parent == 0:
        return root
    else:
        f(root.parent, phrases, graph)
        
def make_sentiment_ptree(root, phrases, graph):
    if len(phrases) == 0:
        print "error: ran out of phrases"
        return
    if ' ' not in root.phrase:
        f(root.parent, phrases, graph)
    elif root.left == 0:
        root.left = pydot.Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        f(root.left, phrases[1:], graph)
    elif root.right == 0:
        root.right = Node(phrases[0], root, 0, 0)
        edge = pydot.Edge(root.phrase, phrases[0])
        graph.add_edge(edge)
        f(root.right, phrases[1:], graph)
    elif root.parent == 0:
        return root
    else:
        f(root.parent, phrases, graph)

In [11]:
grph = pydot.Dot(graph_type='graph')
rootnode = pydot.Node("Node B", style="filled", fillcolor="green")
root2 = f(root, phr, grph)

error: ran out of phrases


In [14]:
grph.write_png('ex.png')

True